# Introduction
Contains notes on study of various optimisation algorithms for training deep neural networks.
Not intended as a learning resource.

## Theory

### Some notation:
- $\Theta$: The parameter space of a machine learning model.
- $f_{\theta}(\cdot)$ - The model to train. Given a feature vector, it predicts a target variable (see below)
- $\mathcal{D}$: A dataset for supervised learning, formally a set of $n$ pairs, $(X_i,y_i)$, where $X_i$ denotes a feature vector (input variable), and $y_i$ is the corresponding target variable.
- $\widehat{y}_i$ - The model prediction given a feature vector, $X_i$.
- $L(y_i,\widehat{y_i})$ - The loss function (essentially a distance function between actual target variable and that predicted by model.
- $C$ - The cost function, which measures a model's error over a group of training instances ($C = \sum_{i} L(y_i,\widehat{y}_i)$).
- $\nabla_{\theta_{t}}C$ - The derivative of the cost function w.r.t the model parameters at time step $t$. A vector.

**What is an optimiser?** - Our ultimate aim in training a neural network is to find the optimal parameterised function $f_{\theta}$, for a given task. This is reduces to searching over the parameter space for $\theta$ for an optimal set of parameters, $\theta_{opt}$. Given that the parameter space can be millions or billions of dimensions in current neural networks, this search must be performed intelligently. This is the job of our optimisation algorithm. 

### Vanilla gradient descent (VGD)

There are essentially three variants of gradient descent. They are characterised by the number of samples in the training dataset that are used to compute the cost function (which is in turn needed for the gradient vector, and paramater updates). They are as follows:

1. **Batch gradient descent**: Uses *all* the training instances in the training dataset to compute the cost function. Whilst this gives the best possible reflection of model performance, it is expensive to implement for large training datasets. 
2. **Stochastic gradient descent**: The converse of batch gradient descent, stochastic gradient descent uses just *one* training instance to calculate the cost, with this training instance chosen randomly. Whilst cheap to implement, this method leads to very noisy parameter updates (movement over parameter space follows no clear direction over short timescales), due to the inaccuracy rising from modelling model performance across the whole training dataset with its loss on just one training instance.
3. **Mini-batch gradient descent**: An attempt at achieving the best of both worlds with regards to batch/stochastic gradient descent. One chooses a batch size, $m$, and the cost is computed as the average over $m$ randomly chosen training instances.

<center>
    <figure>
        <img src='Images/vanilla_grad_descent.png' width=400 alt='missing' />
        <figcaption>Src: https://towardsdatascience.com/improving-vanilla-gradient-descent-f9d91031ab1d</figcaption>
    </figure>
</center>


However, the following challenges are present in all versions of vanilla gradient descent:
1. **Difficulty in choosing learning rate** - A large learning rate may lead to no convergence, with the optimiser instead just oscillating around some (global/local) minimum. Too small a learning rate runs the risk of getting stuck in local minima, as well as extremely slow convergence (and therefore large training times). 
2. **Non-adaptive learning rate schedules** - One quick fix to problem (1) is to have the learning rate vary as a function of time, or epoch number. This is known as setting a learning rate schedule. Whilst this sometimes works, the learning rate must be set in advance of the training process, meaning they cannot adapt to the datasets characteristics (for example, it would be desirable to have a learning rate that adapts to the magnitude of the gradient).
3.) **Global learning rates** - In the methods given above, one learning rate is applied to all parameters. We would like a learning rate that accounts for this.
4.) **Getting stuck in local minima** - For highly non-convex functions, it is likely that variants of gradient descent will get stuck in suboptimal local minima. We would like to develop techniques that encourage convergence toward global minima in parameter space.


### Momentum:

As hinted above, some directions in parameter space may be considerably steeper than others. This can lead to oscillation without real progress towards local/global minimum when using VGD. The momentum adaptation to gradient descent gives the parameter update some inertia in the direction that it is moving, similar to that which a heavy ball has when rolling down a slope. This is shown in the figure below:

<center>
    <figure>
        <img src='Images/momentum_img.png' width=600 alt='missing' />
        <figcaption></figcaption>
    </figure>
</center>

Formally, we can define the weight update for gradient descent with a momentum adaptation as follows:

\begin{align*}
\theta_{t+1} = \theta_{t} + \Delta \theta_{t} \\
\Delta\theta_t = -\eta \nabla_{\theta_t}C + \alpha \Delta \theta_{t-1}
\end{align*}

In short, there is an additional term that incorporates the direction of the parameter update from the previous step, giving the current parameter update 'momentum' in this direction. $\alpha$ is a user defined hyperparameter.


### Adaptive Momentum (Adagrad):

Adagrad adapts the learning rate to two properties of the training process:
1. The scale of the gradient along directions in parameter space.
2. The elapsed training time. 

As a result, rather than a scalar learning rate, we now have a vector learning rate $\mathbf{\eta}$. The Adagrad weight update for the $i$th parameter, at time $t$ takes the form:

\begin{align*}
\theta_{t+1,i} = \theta_{t,i} - \eta_{t,i} g_{t,i} \\
g_{t,i} = (\nabla_{\theta_{t}}C)_i
\end{align*}

$g_{t,i}$ is just the $i$th component of the gradient vector. <br>
We need to define two further terms. First, we introduce the diagonal matrix $G_t \in \mathbb{R}^{d \times d}$, where $d$ is the dimension of parameter space. The $i$th nonzero element of $G_t$, $G_{t,ii}$ is the sum of the squares of the gradients in the direction of $\theta_{i}$ from time $t=1$ up to time step $t$: $G_{t,ii} = \sum_{j=1}^{t-1}g_{j,i}^{2}$. We can then use this term to scale the learning rate accordingly, yielding:

$$
\eta_{t,i} = \frac{\eta_0}{\sqrt{G_{t,ii} + \epsilon}}
$$

where $\epsilon$ is some small positive number to prevent division by zero errors and $\eta_0$ is a hyperparameter.


### RMSProp
Observe that the adaptive learning rates of the Adagrad optimiser will converge to $0$ in the limit of long training times. This is because the denominator is simply the square root of the accumulated gradients squared from timestep $1$ to $t$, which is a monotonically increasing function.
It is undesirable for the learning rate to tend toward $0$, as this reduces the amount of learning that can be done by the model in the latter stages of training. The RMSprop optimiser looks to solve this by normalising the initial hyperparameter $\eta$ with the average of the squared gradients *over the past $w$ timesteps only*. This is achieved with an *exponentially weighted average* over the gradients so far.

To formalise this, we start with a general form for a decaying, adaptive learning rate (we do this for a single parameter, and thus leave out the parameter subscript $i$ for this example):
$$
\eta_t = \frac{\eta_0}{\sqrt{\alpha_t + \epsilon}} 
$$
Recall that for Adagrad, $\alpha_t = \sum_{j=1}^t g_j^2$, where $g_j$ is the gradient at the $j$ th timestep. <br>
For RMSprop, we add a decay factor to give gradients from previous time steps lower weight compared to the most recent gradients. A mathematical statement of this concept is:
$$\alpha_t = \sum_{j=1}^t \rho^{t-j} \cdot g_j^2$$ 
where $0<\rho<1$. This allows us to give $\alpha_t$ a recursive definition, namely $\alpha_t = \rho \alpha_{t-1} + g_t^2$.

RMSprop uses a slight variation on the formalisation above, with $\alpha_t = \rho \alpha_{t-1} + (1-\rho)g_t^2$. The scaling of $g_t^2$ by $(1-\rho)$ allows the user to give the normalisation factor $\alpha_t$ a long or short term memory;  $\rho \approx 0$ means that the normalisation term is dependent on the most recent gradients, whereas $\rho \approx 1$ leads to a learning rate that is influenced by gradients many time steps ago.

### AdaDelta

The function of a user-defined learning rate $\eta$ is to 'guess' the scale of the loss landscape. Do most valleys and hills in the landscape exist on a scale of 0.01,0.1,1.0,10,100 etc units?

All other optimisation algos presented here are first-order methods - use first derivative of C w.r.t theta only. This messes up gradient units though.

### Adam



### Sources
- https://www.analyticsvidhya.com/blog/2021/10/a-comprehensive-guide-on-deep-learning-optimizers/#:~:text=While%20training%20the%20deep%20learning,loss%20and%20improve%20the%20accuracy.
- https://ruder.io/optimizing-gradient-descent/

## Experiments
Comparing the optimisers above on a simple MNIST image classifier, via their learning curves.

In [2]:
## Imports
import torch;torch.manual_seed(1)
import torchvision

In [8]:
##Data processing
train_batch_size = 64
test_batch_size = 1000
mean = 0.1307; std = 0.3081 #global mean and std of MNIST dataset

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
    (mean,),(std,))])
train_dataset = torchvision.datasets.MNIST('',train=True,
                                     download=True,transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           shuffle=True,
                                           batch_size = train_batch_size)

test_dataset = torchvision.datasets.MNIST('',train=False,
                                          download=True,transform=transform)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           shuffle=True,
                                           batch_size = test_batch_size)
#Then some EDA...                       

In [36]:
##Create the network
import torch.nn as nn
import torch.optim as optimisers

#Creating network of size [input, 128,64,32,16,10]

imgs,targets = next(iter(train_loader))
input_dims = tuple((imgs[0][0,:,:]).size())

class FNN(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        #activation function
        self.ReLU = nn.ReLU(inplace=True)
        
        self.l1 = nn.Linear(input_dims[0]*input_dims[1],128)
        self.l2 = nn.Linear(128,64)
        self.l3 = nn.Linear(64,32)
        self.l4 = nn.Linear(32,16)
        self.l5 = nn.Linear(16,10)
            
        self.softmax = nn.Softmax(10)
        
        
    def forward(self,x):
        
        x = x.view(64,1,input_dims[0]*input_dims[1])
        x = self.ReLU(self.l1(x))
        x = self.ReLU(self.l2(x))
        x = self.ReLU(self.l3(x))
        x = self.ReLU(self.l4(x))
        x = self.ReLU(self.l5(x))
        print(x.size())
        
        probs = self.softmax(x)
        
        return probs
        
        
net = FNN()        

In [6]:
import torch.optim as optim
optimisers = {'SGD':optim.SGD(),
             'SGD_momentum':optim.SGD(),
             'AdaGrad':optim.Adagrad(),
             'RMSprop':optim.RMSprop(),
             'Adadelta':optim.Adadelta(),
              'Adam':optim.Adam()
             }

def optimiser_loss():
    'Return optimiser loss on MNIST dataset for model defined above'
    pass

fig,ax = plt.subplots()

for name,optimiser in optimisers.items():
    loss_data = optimiser_loss(...)
    ax.plot(list(range(epoch_num)),loss_data,legend=name)
    
fig.suptitle('Optimiser learning curves')
    

SyntaxError: invalid syntax (2382874803.py, line 5)